<a href="https://colab.research.google.com/github/SruthiVihitha/Steganography/blob/main/Technical%20Documentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Steganography Appilcation**
**Technical Documentation for Steganography Application**

This technical documentation provides a detailed explanation of the Steganography application, including its functionality, components, and implementation details.

**System Requirements:**

*   Python 3.x
*   Tkinter library
*   PIL (Pillow) library
*   OpenCV library


**Installation:**

*   Ensure Python 3.x is installed on your system.
*   Install the required libraries using pip



In [ ]:
pip install tkinter pillow opencv-python


# **Code Structure**
The application is structured into the following key sections:


*   Imports and Global Variables
*   Helper Functions
*   Main GUI Setup
*   Event Handlers
*   Encoding and Decoding Functions
*   Application Execution

# **Imports and Global Variables:**


The code starts by importing necessary modules and defining global variables used throughout the application.

In [ ]:
from tkinter import *
from tkinter import filedialog
import tkinter as tk
from PIL import Image, ImageTk
import os
import cv2
import hashlib

root = tk.Tk()
root.title("Steganography - Hide a Secret Text Message in an Image")
root.geometry("700x500+150+180")
root.resizable(False, False)
root.configure(bg="#d1b3ff")

filename = ""
img = None
h, w, channels = 0, 0, 0
image_path = ""
password = ""
hashed_password = b''
msg = ""
encrypted_image_path = ""


# **Helper Functions**
*showimage()*

This function allows the user to select an image file and displays it in the GUI.

In [ ]:
def showimage():
    global filename, img, h, w, channels, image_path
    filename = filedialog.askopenfilename(initialdir=os.getcwd(),
                                          title='Select Image File',
                                          filetype=(("PNG file", "*.png"),
                                                    ("JPG File", "*.jpg"), ("All file", "*.txt")))
    img = Image.open(filename)
    img = ImageTk.PhotoImage(img)
    lbl.configure(image=img, width=250, height=250)
    lbl.image = img
    img = cv2.imread(filename)
    h, w, channels = img.shape
    image_path = filename


*get_password()*

This function retrieves the user-entered password and hashes it using SHA-256.

In [ ]:
def get_password():
    global password, hashed_password
    password = password_entry.get()
    print("Entered password:", password)
    hash_object = hashlib.sha256(password.encode())
    hashed_password = hash_object.digest()


*get_message()*

This function retrieves the secret message entered by the user.

In [ ]:
def get_message():
    global msg
    msg = message_entry.get()
    print("Entered secret message:", msg)


# **Encoding and Decoding Functions**
*encode()*

This function encodes the secret message into the image using the least significant bit (LSB) steganography technique.

In [ ]:
def encode():
    global img, msg, hashed_password
    msg += '\x00'
    msg = [ord(char) for char in msg]

    msg_index = 0
    for i in range(h):
        for j in range(w):
            if msg_index < len(msg):
                char = msg[msg_index]
                for k in range(3):
                    img[i, j, k] = (img[i, j, k] & 0xFE) | ((char >> (7 - k)) & 1)
                msg_index += 1
            else:
                break
    return img


*decode(encrypted_image_path, password)*

This function decodes the hidden message from the image using the provided password.

In [ ]:
def decode(encrypted_image_path, password):
    img = cv2.imread(encrypted_image_path)
    if img is None:
        print("Image not found. Check the file path and make sure the image exists.")
        return ""

    height, width, channels = img.shape

    hash_object = hashlib.sha256(password.encode())
    hashed_password = hash_object.digest()

    decoded_msg = []
    char_bits = 0
    bits_count = 0

    for i in range(height):
        for j in range(width):
            for k in range(3):
                char_bits = (char_bits << 1) | (img[i, j, k] & 1)
                bits_count += 1
                if bits_count == 8:
                    decoded_msg.append(char_bits)
                    if char_bits == 0:
                        return ''.join(chr(char) for char in decoded_msg[:-1])
                    char_bits = 0
                    bits_count = 0

    return ''.join(chr(char) for char in decoded_msg)


# **Event Handlers**
*Hide()*

This function triggers the encoding process to hide the message in the image.

In [ ]:
def Hide():
    global msg, img
    get_message()
    get_password()
    encode()
    print("Data hidden successfully!")


*Show()*

This function triggers the decoding process to retrieve the hidden message from the image.

In [ ]:
def Show():
    global password
    get_password()
    secret_message = decode(encrypted_image_path, password)
    if secret_message:
        text1.delete(1.0, END)
        text1.insert(END, secret_message)
    else:
        text1.delete(1.0, END)
        text1.insert(END, "Incorrect password or no hidden message found.")


*save()*

This function saves the image with the hidden message to the disk.

In [ ]:
def save():
    global encrypted_image_path, img
    encrypted_image_path = os.path.join(os.path.dirname(filename), "encryptedImage.png")
    cv2.imwrite(encrypted_image_path, img)
    print(f"Image saved to {encrypted_image_path}")


# **Main GUI Setup**
The following code sets up the GUI elements and layout.

In [ ]:
image_icon = PhotoImage(file="icon.png")
root.iconphoto(False, image_icon)
logo = PhotoImage(file="cyberlogo.png")
Label(root, image=logo, height=70, width=85, bg="#751aff").place(x=15, y=1)
Label(root, text="STEGANOGRAPHY", bg="#d1b3ff", fg="black", font="papyrus 25 bold").place(x=120, y=20)

f = Frame(root, bd=3, bg="black", width=340, height=280, relief=GROOVE)
f.place(x=10, y=80)

lbl = Label(f, bg="black")
lbl.place(x=40, y=10)

frame7 = Frame(root, bd=3, width=320, height=280, bg="white", relief=GROOVE)
frame7.place(x=360, y=80)

frame8 = Frame(root, bd=3, width=320, height=100, bg="white", relief=GROOVE)
frame8.place(x=360, y=80)

Label(frame8, text="Hidden Text:", fg="black",bg="white", font="papyrus 12 bold").place(x=0, y=0)

text1 = Text(frame8, font="palatino 20", bg="white", fg="black", relief=GROOVE, wrap=WORD)
text1.place(x=0, y=20, width=321, height=295)

frame9 = Frame(root, bd=3, width=320, height=100, bg="white", relief=GROOVE)
frame9.place(x=360, y=170)

Label(frame9, text="Secret",bg="white" ,fg="black", font="papyrus 15 bold").place(x=4, y=5)
Label(frame9, text="Message:", fg="black",bg="white", font="papyrus 15 bold").place(x=4, y=40)

frame2 = Frame(root, bd=3, width=250, height=150, bg="white", relief=GROOVE)
frame2.place(x=470, y=185)

message_entry = tk.Entry(frame2)
message_entry.pack()

submit_button = tk.Button(frame2, text="SUBMIT", font="palatino 10 bold", command=get_message, width=27, height=2)
submit_button.pack()

scrollbar1 = Scrollbar(frame2)
scrollbar1.place(x=320, y=0, height=150)

scrollbar1.configure(command=text1.yview)
text1.configure(yscrollcommand=scrollbar1.set)

frame6 = Frame(root, bd=3, width=320, height=95, bg="white", relief=GROOVE)
frame6.place(x=360, y=266)

Label(frame6, text="Password:", fg="black",bg="white", font="papyrus 20 bold").place(x=10, y=5)

frame5 = Frame(root, bd=3, width=300, height=300, bg="white", relief=GROOVE)
frame5.place(x=520, y=280)

password_entry = tk.Entry(frame5, show="*")
password_entry.pack()

submit_button = tk.Button(frame5, text="SUBMIT", font="palatino 10 bold", command=get_password, width=20, height=2)
submit_button.pack()

frame3 = Frame(root, bd=3, bg="#d1b3ff", width=330, height=100, relief=GROOVE)
frame3.place(x=10, y=370)

Button(frame3, text="Open Image", width=10, height=1, font="papyrus
